In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 910.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="byzls1nxmDqJ90npm7F1")
project = rf.workspace("sliit-kuemd").project("cattle-diseases")
version = project.version(2)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cattle-diseases-2 in multiclass:: 100%|██████████| 842/842 [00:00<00:00, 2933.55it/s]


In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor
from torch.utils.data import Dataset
from transformers import AutoImageProcessor
from PIL import Image
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from torch import nn, optim

In [ ]:
def load_labels(folder_paths):
    merged_df = pd.DataFrame()
    for path in folder_paths:
        csv_path = os.path.join(path, "_classes.csv")
        df = pd.read_csv(csv_path)
        df["image_path"] = df.iloc[:, 0].apply(lambda x: os.path.join(path, x))
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    return merged_df

In [ ]:
train_df = load_labels(["/content/cattle-diseases-2/train", "/content/cattle-diseases-2/test"])
valid_df = load_labels(["/content/cattle-diseases-2/valid"])

In [ ]:
from typing import Dict, List, Tuple, Union

class CowDiseaseDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = row['image_path']
        # Convert the label columns to numeric type before creating the tensor
        labels = torch.tensor(row[row.keys()[1:-1]].astype(float).values, dtype=torch.float32)

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        #return image, labels # This line is changed
        return {'pixel_values': image, 'labels': labels} #Return a dictionary

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
train_dataset = CowDiseaseDataset(dataframe=train_df, transform=transform)
valid_dataset = CowDiseaseDataset(dataframe=valid_df, transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=11,
    problem_type="multi_label_classification"
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
     # Evaluation at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir='./logs',  # For storing logs
    logging_steps=10,  # Log every 10 steps
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,                          # The model to train
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # The training dataset
    eval_dataset=valid_dataset,           # The validation dataset
    compute_metrics=None,                 # Add custom metric function if necessary
    tokenizer=None,                       # Tokenizer not needed for images
)

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.


In [ ]:
trainer.train()


Step,Training Loss
10,0.661700
20,0.594500
30,0.526900
40,0.468800
50,0.428600
60,0.400100
70,0.380400
80,0.362900
90,0.347100
100,0.333700


TrainOutput(global_step=820, training_loss=0.23906777733709755, metrics={'train_runtime': 548.9191, 'train_samples_per_second': 23.61, 'train_steps_per_second': 1.494, 'total_flos': 1.0043771961065472e+18, 'train_loss': 0.23906777733709755, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.25516369938850403,
 'eval_runtime': 3.7956,
 'eval_samples_per_second': 49.005,
 'eval_steps_per_second': 3.162,
 'epoch': 20.0}

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import torch
import numpy as np
from torchvision import transforms # Import transforms here

def predict_and_display(trainer, valid_df, class_names):
    """
    Predicts labels for the validation dataset and displays the results.

    Args:
        trainer: The Trainer object used for training.
        valid_df: The Pandas DataFrame containing validation data.
        class_names: A list of class names corresponding to the model's output.
    """

    # Get image paths from the validation DataFrame
    image_paths = valid_df['image_path'].tolist()

    # Create a basic transform for prediction
    predict_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to the model's expected input size
        transforms.ToTensor(),           # Convert to PyTorch tensor
        # Add normalization if used during training
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])


    # Create a dataset for prediction (with basic transform)
    predict_dataset = CowDiseaseDataset(dataframe=valid_df, transform=predict_transform)

    # Run prediction
    predictions = trainer.predict(predict_dataset)

    # Get logits and labels
    logits = predictions.predictions
    true_labels = predictions.label_ids

    # Apply sigmoid and threshold at 0.5 for multi-label classification
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    predicted_labels = (probs > 0.5).astype(int)

    # Display actual vs predicted for each image
    for i in range(len(predicted_labels)):
        actual = [class_names[j] for j in range(len(class_names)) if true_labels[i][j] == 1]
        predicted = [class_names[j] for j in range(len(class_names)) if predicted_labels[i][j] == 1]

        print(f"\n📷 Image: {image_paths[i]}")
        print(f"✅ Actual Labels: {actual}")
        print(f"🧠 Predicted Labels: {predicted}")

# Call the function to predict and display results

In [ ]:
class_names = [
    "BRD", "Bovine", "Contagious", "Dermatitis", "Disease",
    "Ecthym", "Respiratory", "Unlabeled", "Healthy", "Lumpy", "Skin"
]

predict_and_display(trainer, valid_df, class_names)


📷 Image: /content/cattle-diseases-2/valid/istockphoto-1319281522-612x612_jpg.rf.995226a175a0a955fbc33de7f3214ce7.jpg
✅ Actual Labels: ['Healthy']
🧠 Predicted Labels: ['Healthy']

📷 Image: /content/cattle-diseases-2/valid/IMG_20220829_124626_jpg.rf.a354942b646265839d9ed56fa4f580e5.jpg
✅ Actual Labels: ['Lumpy', 'Skin']
🧠 Predicted Labels: ['Lumpy', 'Skin']

📷 Image: /content/cattle-diseases-2/valid/images_jpeg.rf.a5131ece4904d8dcaf77e31b72583785.jpg
✅ Actual Labels: ['BRD', 'Bovine', 'Disease', 'Respiratory']
🧠 Predicted Labels: ['Healthy']

📷 Image: /content/cattle-diseases-2/valid/IMG-20220830-WA0065_jpg.rf.9b091d6fbdde75a3d8e5bb0183ec6340.jpg
✅ Actual Labels: ['Lumpy', 'Skin']
🧠 Predicted Labels: ['Lumpy', 'Skin']

📷 Image: /content/cattle-diseases-2/valid/Ayrshirecattle69_jpg.rf.983d9add00cdf1d9e36b3da4db88068d.jpg
✅ Actual Labels: ['Healthy']
🧠 Predicted Labels: ['Healthy']

📷 Image: /content/cattle-diseases-2/valid/images_jpeg.rf.af6c9a8e3d1e1d5ef6d952be559537f5.jpg
✅ Actual Labe

In [ ]:
model_path = "vit_cow_disease_model"
model.save_pretrained(model_path)

# Initialize and save the feature extractor
from transformers import ViTFeatureExtractor

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
feature_extractor.save_pretrained(model_path)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.


['vit_cow_disease_model/preprocessor_config.json']

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Load the model and feature extractor
model_path = "vit_cow_disease_model"
model = ViTForImageClassification.from_pretrained(model_path)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_path)

# Preprocessing function (adjust as needed)
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")
    return inputs

# Load and preprocess an image
image_path = "/content/lumpy.png"  # Replace with your image path
inputs = preprocess_image(image_path)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Process the output (e.g., apply sigmoid for multi-label classification)
probs = torch.sigmoid(logits).squeeze().cpu().numpy()

# Get predicted class (or classes for multi-label) based on probabilities
predicted_class_index = probs.argmax()  # For single-label
# For multi-label, you might threshold the probabilities
predicted_classes = [i for i, prob in enumerate(probs) if prob > 0.5]

# Print or use the predicted class(es)
print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted classes: {predicted_classes}")

Predicted class index: 9
Predicted classes: [9, 10]


In [52]:
import os
from google.colab import files

# Specify the directory containing your saved model
model_path = "vit_cow_disease_model"

# Create a zip archive of the model directory
!zip -r "{model_path}.zip" "{model_path}"

# Download the zip file
files.download(f"{model_path}.zip")

updating: vit_cow_disease_model/ (stored 0%)
updating: vit_cow_disease_model/model.safetensors (deflated 7%)
updating: vit_cow_disease_model/config.json (deflated 58%)
updating: vit_cow_disease_model/preprocessor_config.json (deflated 46%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
import shutil

model_path = "vit_cow_disease_model"  # Your model directory
drive_path = "/content/drive/MyDrive/vit_cow_disease_model"  # Destination on Drive

shutil.copytree(model_path, drive_path)

'/content/drive/MyDrive/vit_cow_disease_model'